### 아이디어
- 최단경로를 이루는 도로의 수가 많다면, 세금이 늘어났을 때 잠재적으로 최단경로가 아니게 될 가능성이 존재한다.
- 따라서 DP쪽으로 생각이 들었다. 세금의 증감량에 따라 통행료가 비쌌지만 도로수가 적어서 이득이 되는 경우가 생길 수 있으니. 
- `DP[i]`: i개의 도로를 거쳐서 도착점에 도착하는 최소 통행료
  - 초기값을 INF로 둔다.
  - bfs의 level마다 도착점에 도착했는지 확인하고, 도착했다면 DP[i]를 갱신한다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N, M, K = map(int, input().split())
  S, D = map(int, input().split())

  G = [[] for _ in range(N + 1)] 
  for _ in range(M) :
    u, v, w = map(int, input().split())
    G[u].append((v, w))
    G[v].append((u, w))
  
  Q = [int(input()) for _ in range(K)]

  DP = [float("inf")] * (M + 1) #i개의 도로를 통해 도착하는 최단거리
  vis = [False] * (M + 1)
  
  Q2 = [(S, 0)]
  vis[S] = True
  k = 1
  while Q2 : 
    nextQ = []
    while Q2 :
      u, ud = Q2.pop()
      for v, vd in G[u] :
        if vis[v] and v != D: continue #도착점을 제외하고 방문한 적이 있으면 패스. 가중치가 항상 양수이므로 이미 갔던 간선을 다시 갈 필요 없는 것이 보장된다.
        vis[v] = True
        if v == D :
          DP[k] = min(DP[k], ud + vd)
          continue
        nextQ.append((v, ud + vd))
    k += 1
    Q2 = nextQ
  
  ans = []
  ans.append(min(DP))
  for q in Q : #O(KM) 이므로 시간초과
    for n, v in enumerate(DP) :
      if v == float("inf") : continue
      DP[n] += q * n
    ans.append(min(DP))
  
  sys.stdout.write("\n".join(map(str, ans)))

sol()

- 무지성 쿼리를 하려하다가 $O(KM)$ 이 TLE가 날 것을 감지했다. 뭔가 하나 더 보여야 할 것 같다.
  - q가 10 이하라는 점을 자세히 살펴봐야 할 것 같다. 왠지 DP의 어느지점까지만 보면 되지 않을까?

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from bisect import bisect_left
class CHT :
  def __init__(self, A, B, EPS=1e-7) :
    self.A = A
    self.B = B
    self.EPS = EPS
    self.build()
  
  def intersect(self, i, j) :
    return (self.B[j] - self.B[i]) / (self.A[i] - self.A[j])

  def build(self) :
    I, X = [], []    
    order = sorted(range(len(self.A)), key=self.A.__getitem__, reverse=True)
    for i in order:
      while True:
        if not I:
          I.append(i)
          break
        elif abs(self.A[I[-1]] - self.A[i]) < self.EPS :
          if self.B[I[-1]] < self.B[i] :
            break
          I.pop()
          if X: X.pop()
        else:
          x = self.intersect(i, I[-1])
          if X and x <= X[-1] :
            I.pop()
            X.pop()
          else:
            I.append(i)
            X.append(x)
            break
    self.I = I
    self.X = X

  def query(self, x) :
    i = self.I[bisect_left(self.X, x + self.EPS)]
    return self.A[i] * x + self.B[i]

def sol() :
  N, M, K = map(int, input().split())
  S, D = map(int, input().split())

  G = [[] for _ in range(N + 1)] 
  for _ in range(M) :
    u, v, w = map(int, input().split())
    G[u].append((v, w))
    G[v].append((u, w))
  
  Q = [int(input()) for _ in range(K)]

  DP = [float("inf")] * (M + 1) #i개의 도로를 통해 도착하는 최단거리
  vis = {}
  
  Q2 = [(S, 0)]
  k = 1
  while Q2 : 
    nextQ = []
    while Q2 :
      u, ud = Q2.pop()
      for v, vd in G[u] :
        if v != D and (u, v) in vis: continue #이미 방문한적이 있는 간선이면 패스. 가중치가 항상 양수이므로 이미 갔던길을 다시 갈 필요 없는 것이 보장된다.
        vis[u, v] = True
        if v == D :
          DP[k] = min(DP[k], ud + vd)
          continue
        nextQ.append((v, ud + vd))
    k += 1
    Q2 = nextQ
  
  ans = []
  acc = 0
  cht = CHT([*range(M+1)], DP)
  ans.append(min(DP))
  for q in Q :
    acc += q
    ans.append(cht.query(acc))

  sys.stdout.write("\n".join(map(str, ans)))

sol()

- bfs가 뭔가 잘못된 것 같다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from bisect import bisect_left
class CHT :
  def __init__(self, A, B, EPS=1e-7) :
    self.A = A
    self.B = B
    self.EPS = EPS
    self.build()
  
  def intersect(self, i, j) :
    return (self.B[j] - self.B[i]) / (self.A[i] - self.A[j])

  def build(self) :
    I, X = [], []    
    order = sorted(range(len(self.A)), key=self.A.__getitem__, reverse=True)
    for i in order:
      while True:
        if not I:
          I.append(i)
          break
        elif abs(self.A[I[-1]] - self.A[i]) < self.EPS :
          if self.B[I[-1]] < self.B[i] :
            break
          I.pop()
          if X: X.pop()
        else:
          x = self.intersect(i, I[-1])
          if X and x <= X[-1] :
            I.pop()
            X.pop()
          else:
            I.append(i)
            X.append(x)
            break
    self.I = I
    self.X = X

  def query(self, x) :
    i = self.I[bisect_left(self.X, x + self.EPS)]
    return self.A[i] * x + self.B[i]

def sol() :
  N, M, K = map(int, input().split())
  S, D = map(int, input().split())

  G = [[] for _ in range(N + 1)] 
  for _ in range(M) :
    u, v, w = map(int, input().split())
    G[u].append((v, w))
    G[v].append((u, w))
  
  Q = [int(input()) for _ in range(K)]
  DP = [float("inf")] * (M + 1) #i개의 도로를 통해 도착하는 최단거리
  
  vis = {} #간선을 지나가는 최소비용
  Q2 = [(S, 0)]
  k = 1
  while Q2 : 
    nextQ = []
    while Q2 :
      u, ud = Q2.pop()
      for v, vd in G[u] :
        if (u, v) in vis and vis[u, v] < ud: continue #어떤 간선을 지나가는데, 최소비용보다 더 큰 비용이 든다면 패스
        vis[u, v] = ud + vd
        if v == D :
          DP[k] = min(DP[k], ud + vd)
          continue
        nextQ.append((v, ud + vd))
    if k >= M : break
    k += 1
    Q2 = nextQ
  ans = []
  acc = 0
  cht = CHT([*range(M+1)], DP)
  ans.append(cht.query(0))
  for q in Q :
    acc += q
    ans.append(cht.query(acc))

  sys.stdout.write("\n".join(map(str, ans)))

sol()

- 이번엔 MLE. 기하급수적으로 노드 숫자가 많아질 수 있긴 하다..

In [ ]:
import io, os, sys, collections
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = 1e18
def sol() :
  N, M, K = map(int, input().split())
  S, D = map(int, input().split())

  G = [[] for _ in range(N + 1)] 
  for _ in range(M) :
    u, v, w = map(int, input().split())
    G[u].append((v, w))
    G[v].append((u, w))
  
  Q = [int(input()) for _ in range(K)]
  DP = [INF] * (M + 1) #i개의 도로를 통해 도착하는 최단거리
  vis = [[INF] * (N+1) for _ in range(N+1)] #간선을 지나가는 최소비용
  Q2 = [(S, 0)]
  nextQ = []
  k = 1
  while Q2 : 
    while Q2 :
      u, ud = Q2.pop()
      for v, vd in G[u] :
        if v == u : continue
        vis[u][v] = ud
        if v == D :
          DP[k] = min(DP[k], ud + vd)
          continue
        
        if vis[u][v] < ud: continue #어떤 간선을 지나가는데, 최소비용보다 더 큰 비용이 든다면 패스
        nextQ.append((v, ud + vd))
    if k >= M : break
    k += 1
    Q2, nextQ = nextQ, Q2
    nextQ.clear()

  ans = []
  DP2 = collections.deque() #확인할 가치가 있는 값들
  for a, b in enumerate(DP) :
    if b >= INF : continue
    if DP2 and DP2[-1][1] < b : continue #더 기울기가 높은데, 상수항도 크다면 패스
    DP2.append([a, b])
  ans.append(min(DP2, key=lambda x : x[1])[1])

  nextDQ = collections.deque()
  for q in Q: 
    res = INF #현재까지 나온 가장 작은 값
    while DP2 : #상수항이 단조감소하도록 설계
      x = DP2.popleft()
      x[1] += (q * x[0])
      if res <= x[1] : continue
      res = x[1]
      nextDQ.append(x)

    DP2, nextDQ = nextDQ, DP2
    nextDQ.clear()
    ans.append(res)

  sys.stdout.write("\n".join(map(str, ans)))

sol()

- 덱DP인것도 발견했는데 이래도 MLE????

In [ ]:
import io, os, sys, collections
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = 1e18
def sol() :
  N, M, K = map(int, input().split())
  S, D = map(int, input().split())

  G = [[] for _ in range(N + 1)] 
  for _ in range(M) :
    u, v, w = map(int, input().split())
    G[u].append((v, w))
    G[v].append((u, w))
  
  Q = [int(input()) for _ in range(K)]
  DP = [INF] * (M + 1) #i개의 도로를 통해 도착하는 최단거리
  vis = [[INF] * (N+1) for _ in range(N+1)] #간선을 지나가는 최소비용
  Q2 = [(S, 0)]
  nextQ = []
  k = 1
  while Q2 : 
    while Q2 :
      u, ud = Q2.pop()
      for v, vd in G[u] :
        if v == u : continue
        if v == D :
          DP[k] = min(DP[k], ud + vd)
          continue
        
        if vis[u][v] < ud: continue #어떤 간선을 지나가는데, 최소비용보다 더 큰 비용이 든다면 패스
        vis[u][v] = ud
        nextQ.append((v, ud + vd))
    if k >= M : break
    k += 1
    Q2, nextQ = nextQ, Q2
    nextQ.clear()

  ans = []
  DP2 = collections.deque() #확인할 가치가 있는 값들
  for a, b in enumerate(DP) :
    if b >= INF : continue
    if DP2 and DP2[-1][1] < b : continue #더 기울기가 높은데, 상수항도 크다면 패스
    DP2.append([a, b])
  ans.append(min(DP2, key=lambda x : x[1])[1])

  nextDQ = collections.deque()
  for q in Q: 
    res = INF #현재까지 나온 가장 작은 값
    while DP2 : #상수항이 단조감소하도록 설계
      x = DP2.popleft()
      x[1] += (q * x[0])
      if res <= x[1] : continue
      res = x[1]
      nextDQ.append(x)

    DP2, nextDQ = nextDQ, DP2
    nextDQ.clear()
    ans.append(res)

  sys.stdout.write("\n".join(map(str, ans)))

sol()

### 자력솔
- vis를 처리해야 할 구간을 정확히 구분해야 한다... 구체적으론 다음 노드 연결지점을 연결하려는 그 시점에 통일적으로 해야한다.
  - 그걸 못해서 탐색할 노드가 기하급수적으로 늘어난게 원인이였다.
- 무슨 쉐도우복싱을 해댄걸까.. 그냥 쉽게 풀 수 있는 문제였을지도 모른다.

### 개선사항
- 우선 DP의 길이부터 최대 M개가 아닌 N개이다. 잘 생각해보니, 정점의 개수까지만 살펴보면 도착하지, 간선의 개수까지 돌려볼 필요가 있을리가 없다.
- 데이크스트라를 변형하여 `D[v][k]`: `k`개의 경로를 겹쳐서 `v`에 도착하는 최단 거리로 만들 수 있다.
  - 이때, 간선을 힙에 다 넣어두지 말고, 시작점으로부터 이어진 간선만 넣는 것을 시작으로, 탐색하면서 heappush & heappop하는 방식으로 확장해볼 수 있다.
  - 힙에는 (누적 가중치, 도착노드, 경로 수)를 저장한다. 
  - 경로 갱신 하기전에, 지금까지 지나온 경로 수를 살펴보면서($cnt := 1 \to cur$), 현재 가중치 `c` 가 도착 정점 `v`에 도달하는 최단경로보다 더 낮은 경로가 있다면 그 경로는 빠꾸친다.
    - 왜냐하면, 그 경로가 존재하는 이상, 최선의 선택을 해도, 이전에 선택한 경로를 뒤따라갈 뿐이기 때문이다.
  - `cnt` 확인이 끝나면 다익스트라의 경로 갱신을 따라한다. 이때 갱신할 수 있었다면 `cnt+1` 을 적용한 새로운 노드를 heap에 넣는다.